In [13]:
import cv2
import numpy as np
import time
import os
import pandas as pd
import matplotlib.pyplot as plt
import re

## 比较CCL和CT获取孔洞信息的速度

In [14]:
csv_folder = "csv"
data_folder = "../../../Data"
jpg_fd_folder = "jpg/jpg_fd_files"

In [15]:
def ct_get_location(fpath):
    # 读取图像
    img = cv2.imread(fpath)
    area_min_radio = 5000 / 4000
    canvas_size = img.shape[0]
    area_min = area_min_radio * canvas_size

    # 灰度化
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # 二值化
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)

    # 查找轮廓
    start_time = time.time()
    contours, hierarchy = cv2.findContours(thresh,  cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    # 遍历轮廓并输出包围盒位置信息
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > int(area_min):
            x, y, w, h = cv2.boundingRect(contour)
    end_time = time.time()
    # print(f"CT程序运行时间为：{end_time - start_time:.4f} 秒")
    return end_time - start_time




In [16]:
def ccl_get_location(fpath):

    # 读取图像
    img = cv2.imread(fpath)
    area_min_radio = 5000 / 4000
    canvas_size = img.shape[0]
    area_min = area_min_radio * canvas_size
    # 灰度化
    img_mb = cv2.medianBlur(img, 3)
    gray = cv2.cvtColor(img_mb, cv2.COLOR_BGR2GRAY)
    # cv2.imwrite('gray.jpg', gray)
    # stats_dict = {}
    # 二值化
    ret, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    # 使用CCL方法查找连通组件
    # cv2.imwrite('binary.jpg', binary)

    start_time = time.time()
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary, connectivity=4)
    # 遍历每个连通组件，并输出其位置信
    for i in range(1, num_labels):
        area = stats[i, cv2.CC_STAT_AREA] 
        if area > int(area_min):
            x, y, w, h, area = stats[i]
            # stats_dict[num_labels] = [x, y, w, h, area]
            # print(f"连通组件 {i} 的位置为：({x}, {y}), 宽度为 {w}, 高度为 {h}")
    end_time = time.time()
    # result_df = pd.DataFrame.from_dict(stats_dict)
    # result_df.to_csv(os.path.join(csv_folder, "stats_result.csv"),index=True)
    # 输出程序运行时间
    # print(f"CCL程序运行时间为：{end_time - start_time:.4f} 秒")
    return end_time - start_time



In [17]:
def get_time_csv(node_list, degree_list, fd_list, type_list):
    fpath_list = []
    for type in type_list:
        for fd in fd_list:
            for degree in degree_list:
                for num in node_list:
                    for root, dirs, files in os.walk(os.path.join(data_folder, jpg_fd_folder, type, fd)): 
                        for file in files: 
                            if str(num)+"d="+str(degree) in os.path.splitext(file)[0]: 
                                fpath_list.append(os.path.join(root,file))
    # print(len(fpath_list))
    ct_list = []
    ccl_list = []
    path_list = []
    velocity_dict = {}
    for path in fpath_list:
        # print("file name = " + path)
        ct_time = ct_get_location(path)
        ccl_time = ccl_get_location(path)
        path_list.append(path)
        ct_list.append(ct_time)
        ccl_list.append(ccl_time)
    velocity_dict["File Path"] = path_list
    velocity_dict["CT Time"] = ct_list
    velocity_dict["CCL Time"] = ccl_list
    result_df = pd.DataFrame.from_dict(velocity_dict)
    result_df.to_csv(os.path.join(csv_folder, "velocity_result.csv"),index=False)
    return ct_list, ccl_list
    

In [18]:
node_list = [200, 500, 1000, 2000, 3000]
fd_list = ["FA2", "FR", "JIGGLE", "KK"]
type_list = ["Sparse", "Uniform"]
degree_list = [6, 8, 10, 12, 15]
ct_list, ccl_list = get_time_csv(node_list, degree_list, fd_list, type_list)
# draw_speed_line_chart_in_same_degree("Sparse", degree, ct_list, ccl_list)
# draw_speed_bar_in_same_degree("Sparse", degree, ct_list, ccl_list)
